In [1]:
import sys
sys.path.insert(0, '../')

import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from scipy.optimize import fsolve
from Black76 import Black76

In [3]:
df = pd.read_csv('../data/data_fx_vol.csv', sep=';')
df.drop(columns=['Unnamed: 2'], inplace=True)
df

,Strike,IV
0,34.381042,1.565449
1,34.477540,1.564397
2,34.574310,1.563344
3,34.671351,1.562290
4,34.768664,1.561236
...,...,...
995,559.080849,1.184723
996,560.650040,1.185899
997,562.223636,1.187074
998,563.801648,1.188250


## ToDo: Rewrite as a class VolSmile or Volsurface.

In [7]:
T=0.667000
F0=80.155
flag = 'c'
target_delta = 0.25

In [8]:
IV_interp = interp1d(df.Strike, df.IV)

def get_vol_from_delta(
    delta: float
):
    def BS_delta(K, flag):
        return Black76(IV_interp(K)).delta(T, K, F0, flag)

    K_target = fsolve(
        func=lambda x: BS_delta(x, flag) - delta, 
        x0=F0
    )
    return IV_interp(K_target)[0]

def risk_reversal():
    return get_vol_from_delta(0.25) - get_vol_from_delta(0.75)

def butterfly():
    return 0.5 * (get_vol_from_delta(0.25) + get_vol_from_delta(0.75)) - get_vol_from_delta(0.5)

In [9]:
print('25% Delta vol =', get_vol_from_delta(target_delta))

25% Delta vol = 0.4637207954564959


In [10]:
print('Risk reversal =', risk_reversal())
print('Butterfly =', butterfly())

Risk reversal = -0.046435550330068964
Butterfly = 0.01722269654973868
